<a href="https://colab.research.google.com/github/sejal-7/Pdfminer/blob/master/JoiningLetter_Extraction_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# spaCy
!python -m spacy download en_core_web_sm

# nltk
!python -m nltk.downloader words
!python -m nltk.downloader stopwords
!pip install pyresparser

     |████████████████████████████████| 12.0 MB 14.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 4.2 MB 16.2 MB/s 
     |████████████████████████████████| 1.5 MB 38.3 MB/s 
     |████████████████████████████████| 

In [ ]:
!pip install PyPDF2

     |████████████████████████████████| 77 kB 4.2 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=0efdecadf45531d44b1ce322f07a4fad91f2cf4e4a371f03681fcceda06ef418
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


Extract PDF to text

In [123]:
import PyPDF2

# Extracting text from PDF

def extractPDF (filePath):
  pdfFileObj = open(filePath, 'rb')
  content = ""
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  numberOfPages = pdfReader.numPages
  for pageNum in range(0,1):
    page = pdfReader.getPage(pageNum)
    content += page.extractText()
  contents = ''.join(content.split('\n'))
  return contents

Supporting Functions to extract Data

In [ ]:
def replace_canadian_period(mail):
    mail = mail.replace(u"\u1427", ".")
    return mail
def replace_fancy_hyphens(mail):
    hlist = [u"\u002d", u"\u058a", u"\u058b", u"\u2010",   u"\u2011", u"\u2012", u"\u2013", u"\u2014", u"\u2015", u"\u2e3a", u"\u2e3b", u"\ufe58", u"\ufe63", u"\uff0d"]
    for h in hlist:
        mail = mail.replace(h, "-")
    return mail

In [ ]:
def lexical_processor(mail):
    mail = replace_canadian_period(mail)
    mail = replace_fancy_hyphens(mail)
    return mail

In [ ]:
import spacy
nlp = spacy.load("en") #we load the English models once, use it many times. Don't include this line into any methods, it'll dramatically reduce the efficiency
def email_preprocessor(mail):
    entities = {}
    mail = lexical_processor(contents)     #mail is unicode
    #mail = cut_greetings(email)        #mail is unicode
    doc = nlp(mail)                    # doc is of spacy.doc type
    entities['emails'] = extract_emails(doc)

In [ ]:
def extract_emails(doc):
    resultlis = []
    for token in doc:
        if token.like_email:
            resultlis.append((token.text))
    return resultlis

In [ ]:
def findDesignation (persons):
  designations = ["Software Engineer Intern", "Graduate Engineer Trainee", "Software Engineer", "Application Developer", "Project Intern"]
  for person in persons:
    if person in designations:
      return person

Extract Basic Information from Extracted Data

In [ ]:
import spacy
import re
from spacy import displacy
from spacy.matcher import Matcher

def extractInformation (contents):
  NER = spacy.load("en_core_web_sm")
  texts = NER(contents)

  data = {}
  data.setdefault('DESIGNATION', [])
  data.setdefault('CITY', [])
  persons = []
  organizations = {""}

  for word in texts.ents:
   if word.label_ == 'PERSON':
     persons.append(word.text)
   if word.label_ == 'ORG':
     organizations.add(word.text)
   if word.label_ == 'GPE':
     data['CITY'].append(word.text)

  desig = findDesignation(persons)
  print(persons)
  data['DESIGNATION'] = [desig]
  persons.remove(desig)
  data['PERSON'] = persons

  data['ORG'] = list (organizations)

  nlp = spacy.load("en_core_web_sm")
  doc = nlp(contents)

  # Email matcher

  emails = extract_emails(doc)
  data['EMAIL'] = emails

  #Phone Number Matcher \d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}

  matcher = Matcher(nlp.vocab)
  pattern = [{'LIKE_NUM': True}, {'LOWER': {'REGEX' : '\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}'}}]
  matcher.add('PHONE', None, pattern)

  matches = matcher(doc)
  data['PHONE'] = [doc[start:end] for match_id, start, end in matches]

  #Salary matcher
  salary_start = re.compile(r'\ INR | Rs.')
  salary_init = salary_start.findall(contents)
  #print(salary_init)
  if salary_init[0] in [' Rs.']:
    salary_pattern = re.compile(r' Rs.+/-')
    salaries = salary_pattern.findall(contents)
    data['SALARY'] = salaries
  elif salary_init[0] in [' INR ']:
    salary_pattern = re.compile(r'\ INR [0-9,\.]*')
    salaries = salary_pattern.findall(contents)
    data['SALARY'] = salaries

  # date matcher

  date_pattern = re.compile(r'((January|February|March|April|May|June|July|August|September|October|November|December)\s([\d]{4}|[\d]{2}))')
  dates = date_pattern.findall(contents)
  data['JOINING DATE'] = dates

  return data


In [ ]:
data

{'CITY': ['Maharasitra', 'India'],
 'DESIGNATION': ['Software Engineer Intern'],
 'EMAIL': ['sejalace7@gmail.com', 'soubhik.das@manastik.com'],
 'JOINING DATE': [('July 2021', 'July', '2021')],
 'ORG': ['',
  'Manastik',
  'Non-DisclosureAgreement("Agreement")containingManastik TechnologiesPvt',
  'the Technical Architect',
  'soubhik.das@manastik.com',
  'CloudPlatforms',
  'Company',
  'INR',
  'Android/Mobile Applications & Technologies'],
 'PERSON': ['Welcometo Manastik'],
 'PHONE': [+91 9284008178],
 'SALARY': [' INR 2,000']}

Write Extracted data to CSV file

In [ ]:
import csv

def writeToCSV (data):
  data_file = open('output.csv', 'w', newline='')
  csv_writer = csv.writer(data_file)

  # Writing data into csv file
  header = data.keys()
  csv_writer.writerow(header)
  csv_writer.writerow(data.values())   # Sejal Resume      <-- data
  print("Data inserted successfully...!  :)")
  data_file.close()

Main Function - to start execution

In [125]:
# Extract data from resume
extracted_text = extractPDF('/content/Manastik_Offer letter_Sejal.pdf')

extracted_Data = extractInformation(extracted_text)
# Write extracted data to CSV file
writeToCSV(extracted_Data)

['Welcometo Manastik', 'Software Engineer Intern']
Data inserted successfully...!  :)
